# Multivariate Cryptography
# Introduction to Quantum-Safe Cryptography (IBM Zurich)
# Simona Samardjiska
# Programming assignment

# Simple iterative reconciliation attack 

In [1]:
load("UOVconstruction.sage") 

######################################
# helper

In [2]:
def SplitInto_k(L, k):
    l = len(L)
    m = l // k # the length of the sublists
    return [list(L[i*m:(i+1)*m]) for i in range(k)]

def AppendIndependent(L, k):
    l = len(L)
    aug_list = [[0 for j in range(k)]+L[i] for i in range(l)]
    for i in range(l):
        aug_list[i][i]=1
    return aug_list

In [3]:
#############################################
##### Reconciliation attack #################

In [4]:
def MakeSystem(oil_vectors, PublicKey, PublicKeySymm):
	system=[]
	for j in range(len(oil_vectors)):
		system += MQeval(PublicKey,Matrix(R,n,1,oil_vectors[j]), Matrix(R,n,1,oil_vectors[j]))
		for k in range(j+1,len(oil_vectors)):
			system += MQeval(PublicKeySymm,Matrix(R,n,1,oil_vectors[j]), Matrix(R,n,1,oil_vectors[k]))
	# print("system")
	# print(system)
	return system

In [5]:
def SolveSystem(system):
	I=ideal(system)
	gr=I.groebner_basis()
	# print("gr")
	# print(gr)
	solution_split=[]
	if len(gr)==c*v:
		solution=[x[i]-gr[i] for i in range(c*v)]
		print("oil vectors found")
		solution_split = SplitInto_k(solution, c)
		print(solution_split)
		print(" ")
	else:
		print("NO oil vectors found")
		if len(gr)==1:
			print("Needs randomization")
		else:
			print("Needs c+=1")
	return solution_split

In [6]:
def FindOilSpace(Public_Key, Public_Key_Symm):
	found=0
	c=ceil(2*n/m-2)
	print("c=",c)
	R = PolynomialRing(K,'x', c*v, order='degrevlex')
	x = R.gens()
	var_oil=SplitInto_k(x, c)
	
	solution_split=[]
	oil_list = []
	recoveredOil = []

	while found < m:
		R = PolynomialRing(K,'x', c*v, order='degrevlex')
		x = R.gens()
		# var_oil=SplitInto_k(x, c)

		###### change the ring of operation
		Public_Key= [M.change_ring(R) for M in Public_Key]
		Public_Key_Symm= [M.change_ring(R) for M in Public_Key_Symm]
		# Central_Map= [M.change_ring(R) for M in Central_Map]
		# Central_Map_Symm= [M.change_ring(R) for M in Central_Map_Symm]
		# S.change_ring(R)

		oil_list +=  solution_split 
		oil_vectors=AppendIndependent(oil_list + var_oil, m)
		print("oil_vectors")
		print(oil_vectors)
		
		system = MakeSystem(oil_vectors, Public_Key, Public_Key_Symm)
		# print("system")
		# print(system)
		solution_split = SolveSystem(system)
		found += len(solution_split)
		if found > 0: c=1
		
	oil_list +=  solution_split 
	recoveredOil = AppendIndependent(oil_list, m)
	return recoveredOil

In [7]:
################ main ##############
q=4
n=8
m=4
v=n-m
c=ceil(2*n/m-2)
# c=m # for full reconciliation in one round
# c determines the number of oil vectors we look for in the beginning
# then c can be set to 1, but also keeping the same c (or extending to the full oil space) is ok, with same asymptotic complexity
# print("c=",c)

K = GF(q)
R = PolynomialRing(K,'x', c*v, order='degrevlex')
x = R.gens()
basis_Fn = (K**n).basis()

In [8]:
###### key generation and signature check #######
Public_Key, S, Central_Map = Keygen(q,n,m)
hash = Matrix(K,m,1,[K.random_element() for j in range(m)])
print("Hash of message: ", hash.list())
signature = Sign(hash, S, Central_Map)
print("Signature: ", signature.list())
verification = Verify(hash, signature, Public_Key)
print("Signature verification: ", verification)    

##### for checking of result of reconciliation attack, otherwise not needed     
Public_Key_Symm = [UpperToSymmetric(M) for M in Public_Key]
Central_Map_Symm = [UpperToSymmetric(M) for M in Central_Map]
O = (S.inverse() * Matrix(K, [basis_Fn[i] for i in range(n-m,n)]).transpose()).transpose()
SpaceO=O.echelon_form()
print("Secret oil space")
print(SpaceO)
###############

Hash of message:  [0, 0, 0, z2]
Signature:  [1, 1, z2 + 1, 1, z2 + 1, 1, z2, z2]
Signature verification:  True
Secret oil space
[     1      0      0      0 z2 + 1 z2 + 1      1     z2]
[     0      1      0      0     z2     z2     z2     z2]
[     0      0      1      0 z2 + 1 z2 + 1      0      0]
[     0      0      0      1      0      0     z2      0]


In [9]:
############## find Oil space #######################
recoveredOil = FindOilSpace(Public_Key, Public_Key_Symm)
print("The recovered oil space is")
print(recoveredOil)
    
    

c= 2
oil_vectors
[[1, 0, 0, 0, x0, x1, x2, x3], [0, 1, 0, 0, x4, x5, x6, x7]]
oil vectors found
[[(z2 + 1), (z2 + 1), 1, z2], [z2, z2, z2, z2]]
 
oil_vectors
[[1, 0, 0, 0, (z2 + 1), (z2 + 1), 1, z2], [0, 1, 0, 0, z2, z2, z2, z2], [0, 0, 1, 0, x0, x1, x2, x3], [0, 0, 0, 1, x4, x5, x6, x7]]
oil vectors found
[[(z2 + 1), (z2 + 1), 0, 0], [0, 0, z2, 0]]
 
The recovered oil space is
[[1, 0, 0, 0, (z2 + 1), (z2 + 1), 1, z2], [0, 1, 0, 0, z2, z2, z2, z2], [0, 0, 1, 0, (z2 + 1), (z2 + 1), 0, 0], [0, 0, 0, 1, 0, 0, z2, 0]]
